In [10]:
import tweepy
import pandas as pd
import re
from sqlalchemy import create_engine

In [4]:
auth = tweepy.OAuthHandler('' , '' )
auth.set_access_token('-', '')
api = tweepy.API(auth)

In [7]:
#return the maximum number of tweets from the timeline 

def data_set(api_call):
    tweet_data = []
    for page in api_call: 
        for tweet in page:
            tweet_data.append(tweet)
    return tweet_data

all_Naval_tweets = data_set(tweepy.Cursor(api.user_timeline, screen_name='Naval', count=200, 
                                          tweet_mode='extended', include_entities=True, include_rts = True).pages())
print("Number of tweets extracted: {}.\n".format(len(all_Naval_tweets)))

Number of tweets extracted: 3215.



In [8]:
#one tweet can have multiple hashtags. Postgres can unnest arrays, so hashtags need list as their data type 

hashtag_list = []
for tweet in all_Naval_tweets:
#if no hashtag, return an empty list (list is default data type for empty Tweet entities in Tweepy api)    
    if len(tweet.entities['hashtags']) == 0:
        hashtag_list.append(tweet.entities['hashtags'])
#if one hashtag in the tweet, take text of this single hashtag
    elif len(tweet.entities['hashtags']) == 1:
        hashtag_string = tweet.entities['hashtags'][0]['text']
#before appendding to the list, convert the string to a list        
        hashtag_list_function = hashtag_string.split(",")
        hashtag_list.append(hashtag_list_function)   
#if a tweet contains more than 1 tweet, append all text elements to a list and then append this list to the main list          
    elif len(tweet.entities['hashtags']) > 1:
        empty_hashtag_list = []
        for hashtag in tweet.entities['hashtags']:
            empty_hashtag_list.append(hashtag['text'])
        hashtag_list.append(empty_hashtag_list)

In [11]:
#a tweet can contain user mentions and urls. Remove both from the tweet text.  
#also remove extra whitespaces and linebreaks to keep a clear text field 

tweet_text_list = []
for tweet in all_Naval_tweets:
    remove_user_mentions = re.sub(r"@\S+", '', tweet.full_text)
    remove_urls = re.sub(r'http\S+', '', remove_user_mentions)
    remove_RT = remove_urls.replace('RT', '') 
    remove_intra_whitespace = re.sub(' +', ' ', remove_RT)
    remove_trailing_whistespace = remove_intra_whitespace.lstrip()
    remove_line_breaks = remove_trailing_whistespace.replace('\n','')
    tweet_text_list.append(remove_line_breaks)

In [12]:
#some tweets contain retweets. For every tweet, append either the retweeted user data, or append 'None'
#whether a tweet is a retweet is determined by whether a tweet starts with 'RT'

retweet_user_id_list = []
retweet_user_name_list = []
retweet_user_screen_name_list = []
retweet_user_location_list = []
retweet_user_description_list = []
retweet_user_created_timestamp = []
retweet_user_followers_count = []

for tweet in all_Naval_tweets:
    if tweet.full_text.startswith("RT @"): 
        retweet_user_id_list.append(tweet.retweeted_status.user.id)
        retweet_user_name_list.append(tweet.retweeted_status.user.name)
        retweet_user_screen_name_list.append(tweet.retweeted_status.user.screen_name)
        retweet_user_location_list.append(tweet.retweeted_status.user.location)
        retweet_user_description_list.append(tweet.retweeted_status.user.description)
        retweet_user_created_timestamp.append(tweet.retweeted_status.user.created_at)
        retweet_user_followers_count.append(tweet.retweeted_status.user.followers_count)
    else:
        retweet_user_id_list.append(None)
        retweet_user_name_list.append(None)
        retweet_user_screen_name_list.append(None)
        retweet_user_location_list.append(None)
        retweet_user_description_list.append(None)
        retweet_user_created_timestamp.append(None)
        retweet_user_followers_count.append(None)

In [13]:
#create a boolean value to determine whether a tweet is a retweet

is_retweeted_list = []
for tweet in all_Naval_tweets:
    if tweet.full_text.startswith("RT @"): 
        is_retweeted_list.append(1)
    else:
        is_retweeted_list.append(0)

In [14]:
#create a boolean value to determine whether a tweet has at least one hashtag

has_hashtag_list = []
for tweet in all_Naval_tweets:
    if len(tweet.entities['hashtags']) > 0:
        has_hashtag_list.append(1) 
    else:
        has_hashtag_list.append(0)     

In [15]:
#create a dataframe and insert columns with tweet related data

df_tweets = pd.DataFrame()

df_tweets['tweet_id'] = [tweet.id for tweet in all_Naval_tweets]
df_tweets['created_timestamp'] = [tweet.created_at for tweet in all_Naval_tweets]
df_tweets['tweet_text'] = tweet_text_list
df_tweets['hashtags'] = hashtag_list
df_tweets['retweet_count'] = [tweet.retweet_count for tweet in all_Naval_tweets]
df_tweets['favorite_count'] = [tweet.favorite_count for tweet in all_Naval_tweets]
df_tweets['is_retweeted'] = is_retweeted_list
df_tweets['has_hashtags'] = has_hashtag_list
df_tweets['tweet_user_id'] = [tweet.user.id for tweet in all_Naval_tweets]
df_tweets['retweet_user_id'] = retweet_user_id_list

In [16]:
#create a dataframe and insert columns with user related data

df_tweet_users = pd.DataFrame()

df_tweet_users['user_id'] = [tweet.user.id for tweet in all_Naval_tweets]
df_tweet_users['user_name'] = [tweet.user.name for tweet in all_Naval_tweets]
df_tweet_users['screen_name'] = [tweet.user.screen_name for tweet in all_Naval_tweets]
df_tweet_users['user_location'] = [tweet.user.location for tweet in all_Naval_tweets]
df_tweet_users['description'] = [tweet.user.description for tweet in all_Naval_tweets]
df_tweet_users['created_timestamp'] = [tweet.user.created_at for tweet in all_Naval_tweets]
df_tweet_users['followers_count'] = [tweet.user.followers_count for tweet in all_Naval_tweets]

df_retweet_users = pd.DataFrame()

df_retweet_users['user_id'] = retweet_user_id_list
df_retweet_users['user_name'] = retweet_user_name_list
df_retweet_users['screen_name'] = retweet_user_screen_name_list
df_retweet_users['user_location'] = retweet_user_location_list
df_retweet_users['description'] = retweet_user_description_list
df_retweet_users['created_timestamp'] = retweet_user_created_timestamp
df_retweet_users['followers_count'] = retweet_user_followers_count

#user data is a combination of both users who have been retweeted and the tweeting user 
df_users_by_tweet = pd.concat([df_tweet_users, df_retweet_users])
#remove duplicate users in the dataframe 
df_users_1 = df_users_by_tweet.drop_duplicates(subset=['user_id'])
df_users = df_users_1.dropna(subset=['user_id']) 

In [17]:
#set up the connection to the Postgres database 

dbschema='raw' 
engine = create_engine(
    'postgresql://postgres:@localhost:5432/postgres',
    connect_args={'options': '-csearch_path={}'.format(dbschema)})
engine.connect()

/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [18]:
#truncate existing tables to avoid any duplicates and to start with a clean dataset

with engine.begin() as conn:
    conn.execute('''TRUNCATE TABLE users CASCADE''')
    conn.execute('''TRUNCATE TABLE tweets CASCADE''')

In [19]:
#insert the dataframes to the tables created in Postgres

df_users.to_sql('users', engine,if_exists='append', index=False)
df_tweets.to_sql('tweets', engine, if_exists='append', index=False)